# UDP Encryption and Decryption with Two Alveo U280s in OCT 

In this notebook, we are demonstrating a simple example of how two Alveo U280s in OCT can talk to each other. We make use of Xilinx CMAC and UDP network layer IPs as the networking infrastructure and add sender/receiver user logic on top of that. We have also added the option for encryption and decryption if needed. The sender FPGA reads a text file from the host memory, encrypts if required before sending through the network. On the other end, the receiver FPGA receives the packet decrypts if required and delivers it to the host. 

Note: It is assumed that you have already generated the bitstream that is required to run this application.

In [ ]:
import time

## Sender/Receiver Parameters

In [ ]:
remote_host_sender = '<node>.cloudlab.umass.edu' #eg: pc155.cloudlab.umass.edu
remote_host_receiver = '<node>.cloudlab.umass.edu'
remote_host_username = '<username>'
private_key = '<private key>'
dest_dir = '/users/' + remote_host_username + '/udp-demo'

In [ ]:
bitstream = 'udp_demo_if0.xclbin'
num_packets = 1
encrypt = True # (False if encryption is not required)
decrypt = True # (False if decryption is not required)
sender_ip = '<sender IP>'
receiver_ip = '<receiver IP>'
ip_gateway = '<IP gateway>'

## Verify the SSH Connection

In [ ]:
!ssh -o StrictHostKeyChecking=no -i {private_key} {remote_host_username}@{remote_host_sender} "echo \"Successfully connected to {remote_host_sender}\" && mkdir -p {dest_dir}"

In [ ]:
!ssh -o StrictHostKeyChecking=no -i {private_key} {remote_host_username}@{remote_host_receiver} "echo \"Successfully connected to {remote_host_receiver}\" && mkdir -p {dest_dir}"

## Copy Files

In [ ]:
!scp -i {private_key} ../build_hw_if0/udp_demo_if0.xclbin sender.py validate.py vnx_utils.py ../host/alice29.txt {remote_host_username}@{remote_host_sender}:{dest_dir}

In [ ]:
!scp -i {private_key} ../build_hw_if0/udp_demo_if0.xclbin receiver.py validate.py vnx_utils.py {remote_host_username}@{remote_host_receiver}:{dest_dir}

## Sender/Receiver Commands

In [ ]:
dest_dir_cmd = f"cd {dest_dir};"
# Rx command
xrt_cmd = 'source /opt/xilinx/xrt/setup.sh;'
rx_params = f"{bitstream} --num_packets {num_packets} --decrypt {decrypt} --my_ip {receiver_ip} --their_ip {sender_ip} --ip_gw {ip_gateway}"
rx_run_cmd = f"python3.8 receiver.py {rx_params}"
rx_buf_del = 'rm -f rx_buf.txt;'
rx_cmd = 'nohup /bin/bash -c ' + f"'{dest_dir_cmd} {rx_buf_del} {xrt_cmd} {rx_run_cmd}'" + f" > {dest_dir}/rx_out.log 2>&1 &"
# Tx command
tx_params = f"{bitstream} --num_packets {num_packets} --encrypt {encrypt} --my_ip {sender_ip} --their_ip {receiver_ip} --ip_gw {ip_gateway}"
tx_run_cmd = f"python3.8 sender.py {tx_params}"
tx_cmd = 'nohup /bin/bash -c ' + f"'{dest_dir_cmd} {xrt_cmd} {tx_run_cmd}'" + f" > {dest_dir}/tx_out.log 2>&1 &"

### Receiver

In [ ]:
!ssh -i {private_key} {remote_host_username}@{remote_host_receiver} "{rx_cmd}"

In [ ]:
time.sleep(5)

### Sender

In [ ]:
!ssh -i {private_key} {remote_host_username}@{remote_host_sender} "{tx_cmd}"

In [ ]:
time.sleep(5)

## Retrieve the Text File

In [ ]:
!rm -f ./rx_buf.txt
!scp -i {private_key} {remote_host_username}@{remote_host_receiver}:{dest_dir}/rx_buf.txt .

In [ ]:
file_path = './rx_buf.txt'
with open(file_path, 'r') as file:
    content = file.read()
print(content)